In [2]:
import re
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from tqdm import tqdm

# page nos. 1 to 574
telugu_url = 'https://dsal.uchicago.edu/cgi-bin/app/gwynn_query.py?page='

In [3]:
soups = {}

def get_soup(page):
    if page in soups:
        return soups[page]
    
    res = requests.get(telugu_url + str(page))
    soup = BeautifulSoup(res.text)
    soups[page] = soup
    return soup


In [4]:
for i in tqdm(range(1,575)):
    get_soup(i)

100%|██████████| 574/574 [04:46<00:00,  2.00it/s]


In [5]:
import pickle
pickle.dump(soups, open('./telugu_soups.pickle', 'wb'))

RecursionError: maximum recursion depth exceeded

In [6]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', None)

In [7]:
def process_entry(div):
    return {
        'telugu': div.select_one('te').text.strip(),
        'transliteration': ''.join(s.strip() for s in div.select_one('hw').children if type(s) == NavigableString),
        'subentry_count': len([1 for b in div.select('b') if b.text.strip().isnumeric() or len(b.text.strip().split())>1]),
        'text': div.text.strip()
    }

sorted_entries = pd.concat([pd.DataFrame([process_entry(div) for div in soups[page].select('div.hw_result > div')]) for page in soups.keys()], ignore_index=True).sort_values('subentry_count', ascending=False)

In [22]:
sorted_entries[sorted_entries.subentry_count > 4].sample(10).sort_values('subentry_count')

,telugu,transliteration,subentry_count,text
14292,నిధి,nidhi,5,"నిధి nidhi nidhi n. 1 fund. 2 treasure house, repository. 3 pl. nidhulu reserves. 4 fig. fund or store of good qualities; cadarangamloo ~ expert at chess."
3271,ఇలాకా,ilākā,5,ఇలాకా ilākā ilaakaa n. 1 territory. 2 jurisdiction. 3 belonging; neenu aayanagaari ~ leNDi I am one of his men; aame waaDi ~ loo undi she is being kept by him.
2944,ఆశ్రయం,āśrayam,5,"ఆశ్రయం āśrayam aaśrayam n. 1 refuge, asylum, retreat. 2 place, abode. 3 support, protection, patronage; naaku ~ iccEEDu he gave me protection. 4 dependence."
19884,మరగు,maragu,6,"మరగు maragu maragu v.i. and t. 1 to have a liking or fondness for 2 to have a taste for, be addicted to, be keen on, go in for; sukham ~ to be fond of living at ease, take life easy; pillalloo caduwukonee waaLLu aaTalu maragaranii, aaTalu marigina waaLLu cadawaranii aawiDaki gaTTi nammakam it was her firm belief that children who studied were not keen on games, while those who were keen on games did not study; see ruci ~."
16934,పెంకు,peṅku,6,పెంకు peṅku penku n. 1 tile; maaDu ~ colloq. top of the head. 2 piece of broken pottery or glass. 3 shell of a fruit; welaga ~ shell of a wood apple. 4 (also penka) flat pan for parching grain or for frying.
4345,ఊహ,ūha,6,"ఊహ ūha uuha n. 1 thought, idea, concept. 2 reasoning, inference. 3 guess, conjecture, supposition. 4 imagination, fancy. 5 contrivance. 6 awareness; see graahi."
19198,భావన,bhāvana,6,"భావన bhāvana bhaawana n. 1 thought, thinking, idea, concept; oka pratyeeka paaribhaaSika ~ ku oka bhaaSaloo sarayna maaTa leekapoowaccu a language may not have the right word for a particular technical concept; pekkumandi madhyawargaalawaari ~ loo swaraajyam sukhajiiwanaaniki paryaayapadam aypooyindi in the thinking of many middle class people self-government became synonymous with wellbeing. 2 feeling; preema ~ lu feelings of love. 3 process followed in Ayurvedic medicine of steeping an ingredient many times in the juice of another (e.g., ginger or cummin seed in limejuice)."
16029,పాకు,",pāku, prāku",7,"పాకు, ప్రాకు pāku, prāku paaku1, praaku v.i. 1 to creep, crawl. 2 (of plants) to climb. 3 fig. to spread, extend, find its way to, affect; matawibheedaalu deeśamloo wiwidha praantaalaku paakEEyi religious differences spread to various parts of country; pleeg wyaadhi prapancam antaTaa paakindi plague spread over the whole world; unna peTTubaDulu paTTaNaalaku ~ tunnaayi what capital there is finds its way to the towns; nayraaśyam iiyanaku kuuDaa praakindi despair affected him also."
11485,తిన్నగా,tinnagā,8,"తిన్నగా tinnagā tinnagaa adv. 1 straight, direct[ly]; ~ weLLaNDi keep straight on. 2 honestly, uprightly, fairly, properly; ~ sampaadincu to earn by fair means; ~ uNDu to behave properly. 3 gently, slowly, steadily. 4 (of health) well, all right, normal; mii aaroogyam ~ undaa? is your health all right? or are you keeping well?"
23796,విరుచు,virucu,11,"విరుచు virucu wirucu v.t. 1 to break [off], snap [off]; naa maaTalu madhyaloonee wiriceesEEDu he interrupted what I was saying. 2 to break to pieces. 3 to twist; o.-i ceyyi ~ to twist s.o.'s arm. 4 pedawi ~ to curl o.'s lip in scorn or disapproval. 5 to curdle (milk). 6 to counteract (poison). 7 to separate words from each other; akkaDakkaDa maaTalu tappugaa wirustuu sundaram uttaram antaa cadlwEEDu making mistakes in some places over the separation of words, Sundaram read the whole letter."
